In [ ]:
 ! pip install -q kaggle

from google.colab import files

files.upload()

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets list


In [ ]:
!kaggle datasets download -d emmarex/plantdisease

In [ ]:
!unzip /content/plantdisease.zip

In [ ]:

import numpy as np
import pickle
import cv2
from os import listdir
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt



In [ ]:


Epochs= 14
lr= 1e-3
Batchsize = 32
default_image_size = tuple((128, 128))
image_size = 0
directory_root = '../input/plantdisease'
width=128
height=128
depth=3



In [ ]:
def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None :
            image = cv2.resize(image, default_image_size)   
            return img_to_array(image)
        else :
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [ ]:


image_list, label_list = [], []
try:
    print("[INFO] Loading images ...")
    root_dir = listdir(directory_root)
    print(root_dir)
    for directory in root_dir :
        # remove .DS_Store from list
        if directory == ".DS_Store" :
            root_dir.remove(directory)

    for plant_folder in root_dir :
        plant_disease_folder_list = listdir(f"{directory_root}/{plant_folder}")
        
        for disease_folder in plant_disease_folder_list :
            # remove .DS_Store from list
            if disease_folder == ".DS_Store" :
                plant_disease_folder_list.remove(disease_folder)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{directory_root}/{plant_folder}/{plant_disease_folder}/")
                
            for single_plant_disease_image in plant_disease_image_list :
                if single_plant_disease_image == ".DS_Store" :
                    plant_disease_image_list.remove(single_plant_disease_image)

            for image in plant_disease_image_list[:400]:
                image_directory = f"{directory_root}/{plant_folder}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg") == True or image_directory.endswith(".JPG") == True:
                    image_list.append(convert_image_to_array(image_directory))
                    label_list.append(plant_disease_folder)
    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")
image_size = len(image_list)
print (f"total image processed {image_size} ")



In [ ]:
image_labels = []
count=0
for label in label_list:
    if label.split('_')[len(label.split('_'))-1] == 'healthy':
        count+=1
        image_labels.append(0)
    else:
        image_labels.append(1)

In [ ]:

import numpy as np
image_list = np.array(image_list, dtype=np.float16) 
image_label=np.array(image_labels)
image_list=image_list.reshape(5725,128*128*3)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
x_res, y_res = sm.fit_resample(image_list, image_label)


In [ ]:
np_image_list=x_res.reshape(9546,128,128,3)
image_label=y_res

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(10,10))
for i in range (16):
  n = random.randint(1,5000)
  plt.subplot(4,4,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow((np_image_list[n]).astype(np.uint8))
  plt.xlabel(image_label[n])
plt.show()

In [ ]:
np_image_list/=255

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_label, test_size=0.2,shuffle=True) 

In [ ]:

imageaug = ImageDataGenerator(
    
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True,vertical_flip=True,
    fill_mode="nearest")

In [ ]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D,BatchNormalization,Flatten,Dense,Dropout
model = tf.keras.models.Sequential()
from tensorflow.keras.layers import Activation
inputShape = (height, width, depth)


model.add(Conv2D(32, (3, 3), padding="same",activation='relu',input_shape=inputShape))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding="same",activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), padding="same",activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding="same",activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (3, 3), padding="same",activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding="same",activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512,activation='relu',kernel_initializer='he_normal'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128,activation='relu',kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(1))
model.add(Activation("sigmoid"))



In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.optimizers import Adam
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate=lr, decay=lr/Epochs),metrics=["accuracy"])

In [ ]:
history = model.fit(
    imageaug.flow(x_train, y_train, batch_size=Batchsize),
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) / Batchsize,
    validation_steps=len(x_test)/Batchsize,
    epochs=Epochs, verbose=1
    )

In [ ]:

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
#Train and validation accuracy
plt.plot(epochs, acc, 'b', label='Training accurarcy')
plt.plot(epochs, val_acc, 'r', label='Validation accurarcy')
plt.title('Training and Validation accurarcy')
plt.legend()

plt.figure()
#Train and validation loss
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.show()

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:

import numpy as np
from sklearn.metrics import confusion_matrix 
y_true=[]
for element in y_test:
    y_true.append(element)
prediction_proba= model.predict(x_test)
prediction=[]
for pred in prediction_proba:
  if pred >0.5:
    prediction.append(1)
  else:
    prediction.append(0)
cnf_matrix = confusion_matrix(y_true,prediction)

In [ ]:
cnf_matrix

In [ ]:

import itertools
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
plt.figure(figsize=(5, 5))
plot_confusion_matrix(cnf_matrix, classes=['1' , '0'],normalize=True,
                      title='Confusion matrix, with normalization')
plt.show()

In [ ]:
model.save('CNN_model.keras')